In [1]:
# Imports
from IOHexperimenter import IOH_function, IOH_logger
from IOH_function_wrapper import IOH_function_logger
import input_reader
import parameters
import datetime
from shutil import make_archive
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a dictionary that contains algorithms and splitpoints / targets
data = input_reader.create_dict()

In [3]:
class Switch:
    """ Class to implement the switch between algorithms.

    Parameters:
    ---------------
    func : object, callable
        The function to be optimized.

    budget : int
        The budget that is available for function evaluations.

    data : dict
        A dictionary with tau as key, algorithm as value


    Attibutes:
    ---------------
    None

    """

    def __init__(self, func, budget, data):
        self.func = func
        self.budget = budget * func.number_of_variables
        self.data = data

    def __call__(self):
        """ Runs the switch routine. 
        
        Returns:
        -----------
        best_so_far_variables : array
                The best found solution.

        best_so_far_fvaluet: float
               The fitness value for the best found solution.
        """

        # Initialize parameters
        params = parameters.init_params(self.budget)

        # Iterate through dictionary
        for tau, alg in data.items():
            algorithm = alg(self.func)
            # Set algorithm parameters based on parameters object
            algorithm.set_params(params)

            # Define stopping criteria
            func = self.func
            budget = self.budget

            def stopping_criteria():
                return (func.best_so_far_precision) <= tau or (
                    func.evaluations > budget)

            algorithm.set_stopping_criteria(stopping_criteria)

            # Run algorithm and extract parameters
            algorithm.run()
            params = algorithm.get_params(params)

            if self.func.evaluations >= budget:
                break

        # Return best point and fitness value found by last run algorithm
        return (algorithm.func.best_so_far_variables, algorithm.func.best_so_far_precision, params) 

In [4]:
# Experiment routine
#path = "/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/BFGS-CMAES/Test_scaled_H_on_multiple_usecases"
now = datetime.datetime.now()
experiment_ID = str(now.day) + "-" + str(now.month) + "-" + str(
    now.strftime('%H')) + str(now.strftime('%M')) + str(now.strftime('%S'))
dimensions = [20]
instances = [1, 2, 3, 4, 5]
func_ids = [10]
exp_budget = 20000
runs = 5
switch_combination = "BFGS-CMAES"
settings = "H_as_C_scaling_1e-1_and_multip_2_sigma" + "_f" + str(func_ids[0]) + "_d" + str(dimensions[0])
#experiment_name = path + "/experiment" + experiment_ID + "_" + settings

for func_id in func_ids:
    for dim in dimensions:
        for inst in instances:
            f = IOH_function_logger(func_id, dim, inst, suite='BBOB')
            #log = IOH_logger(experiment_name, "run",
                      #       switch_combination + settings, "switch-routine")
            #f.add_logger(log)
            for run in range(0, runs):
                if run > 0:
                    f.reset()
                    f.x_hist = []
                    f.f_hist = []
                switch = Switch(f, exp_budget, data)
                x_opt, prec, params = switch()
                print(f' run {run} dim{dim} f{func_id} inst{inst} complete, precision: {prec}')
            #f.clear_logger()

#make_archive(experiment_name, "zip", experiment_name)

 BFGS started
BFGS complete
evals: 1302 x_opt: [-1.72614321 -1.50799268 -1.37346766  3.34213669  2.84528505  3.70577754
 -1.7940184   0.81841297 -3.72296137  2.87388819  0.68487398  0.7579742
  2.61335818 -3.46539489  1.35071484  3.92522799  2.05638458 -1.60821873
  2.64452232  2.53981068]


TypeError: Cannot interpret '1' as a data type

In [7]:
switch.func.evaluations, switch.func.best_so_far_variables

(291, array([1.94841689, 2.92095659]))